# 2) Learning Theory

* Supervised learning (briefly)
* Theory of generalization 
    * Hoeffding inequality
    * Dichotomies, Growth func., Breakpoints
    * VC dimensions
* Bias variance tradeoff
* Regularization and Validation

### Supervised learning

In the Supervised Learning setting we are given a dataset $D = \{(x_1,y_1),...,(x_n,y_n)\}$ to learn from. The way we "learn" is by using D to search a hypothesis space H for some "best" hypothesis $g\in H$. By "best", we mean that we want g to be as close to the __ unknown target function f__ as possible, i.e.: 

$$ g \approx f $$

We most often find g by minimizing an in-sample error function $E_{in}(h)$. The process of finding g is what we call "training" and when we have found it, we can use it to predict on new inputs.

Within supervised learning we distinguish between __two subcategories__. If the output y is a real number we call it __regression__. If the output y is a member of a discrete set - e.g. $y \in \{red,blue,green\}$ - then we call it __classification__.

In Supervised Learning we try to minimize $E_{in}(g)$, but what we _really_ care about in the end is having a low out-of-sample error $E_{out}(g)$, that is, in the end we only care about how g performs on _new_ data. Since we cannot measure $E_{out}(g)$ during training, we hope that we can generalize such that $E_{in}(g)$ - which we _can_ measure - is close to $E_{out}(g)$. 


### Learning theory

The big question in learning theory, is whether learning is even feasible - can we learn something about an entire target function/distribution based only on a finite data set?


1) Can we make sure $E_{out}(g)$ is close to $E_{in}(g)$? __(generalize)__

2) Can we make $E_{in}(g)$ small? 

The next section will show that yes, it is possible to generalize from a limited sample D to the infinite space X - but only in a probabilistic sense. The second question we can answer only after tranining and observing the value of $E_{in}(g)$. If $E_{in}(g)$ too big, then perhaps the target function is too complex, or maybe too random. 

The theory of generalization is different depending on the setting. Below we discuss the theory in a binary classification setting. And the section called "Bias Variance tradeoff" will be about the theory in a regression setting.

### Theory of generalization (binary classification)

#### Hoeffding inequality 

The Hoefding inequality is the starting point for the discussion about generalization in a binary classification setting. It tells us something about how a random sample _tends_ to be similar to the whole space as the size of the sample grows. It's important that the sample is picked __random and independently according to some probability distribution P(x)__ - if it's not then we lose the benefit of probabilistic analysis. The inequality looks like this:

$$P[\rvert E_{in}(h) - E_{out}(h)\rvert > \epsilon] \le 2e^{-2\epsilon^2N}$$

In words it states that the probability that the difference between $E_{in}$ and $E_{out}$ is greater than some epsilon, is less than, or equal to $2e^{-2\epsilon^2N}$. $E_{in}$ being too far from $E_{out}$ is bad, and we would like to say that this probability is low. The inequality tells us how many datapoints (the N) we need to keep the probability low for a given tolerance epsilon. The important thing to notice is the $\epsilon^2$ and the N in the negative exponent, which tries to cancel each other out. Since epsilon is squared, we need N to become much bigger once we decrease epsilon.

For __example__, if we can accept a maximum $10\%$ chance that $\rvert E_{in}-E_{out} \rvert > 0.05$ , then we can solve the following for N: 

$$0.1 \ge 2e^{-2(0.05)^2N} \implies N > 599$$

Which mean we need more than 599 datapoints to guarantee this chance. If we want the chance to be less than $5\%$, we need at least 739 samples.


##### Verification vs learning

Another important think to realize is that the hoeffding bound as stated above is __valid for a fixed hypothesis h__. But if we only have one hypothesis, we are just verifying if its good or not. __The hoeffding inequality is valid under the assumption that h is fixed before generating the data set D__. But in learning, g is not fixed ahead of time, because which hypothesis becomes g depends on the data, i.e. it is chosen after the data is generated. To use the inequality for learning with multiple h's, we must change it abit.

What we would like to state after training is: "$P[\rvert E_{in}(g) - E_{out}(g) \rvert > \epsilon]$ is small", where g is the hypothesis we report after training. To do this we must bound $P[\rvert E_{in}(g) - E_{out}(g \rvert > \epsilon]$ in a way that does not depend on which g the algorithm reports. We do this the following way.

Since g is one of M hypothesis in the set (Assume for now that M is finite), the following holds:

$$ \rvert E_{in}(g) - E_{out}(g) \rvert > \epsilon \implies \rvert E_{in}(h_i) - E_{out}(h_i) \rvert > \epsilon  $$ 

For some $h_i$. Now the $h_i$'s are fixed and we can apply the following rules about events $B_i$:

1) if $B_1 \implies B_2$ then $B_1 \le B_2$

2) $P[B_1\ \mathrm{or}\ B_2 \ \mathrm{or} \ ... \ \mathrm{or} \ B_M] \le  P[B_1] + P[B_2] +...+ P[B_M]$

To get:

$$P[\rvert E_{in}(g) - E_{out}(g) \rvert > \epsilon] \le \sum_{m=1}^M P[\rvert E_{in}(h_m) - E_{out}(h_m) \rvert > \epsilon]$$

Now we can apply the standard hoeffding bound to each term, and get the new bound:

$$P[\rvert E_{in}(g) - E_{out}(g)\rvert > \epsilon] \le 2Me^{-2\epsilon^2N}$$

This final bound is a factor of M looser than the vanilla hoeffding bound. This is very loose since the bad events are hugely overlapping (many hypothesis are very similar to one another, and thus if one is bad, many other are likely to be aswell). Furthermore __it is meaningless for inifinite size hypothisis sets - which is most often the case__. In the following we seek to replace M with something meaningful in the case of a infinite sized hypothesis set.

__The hoeffding bound gives the answer to the first of the two questions regarding feasibility of learning__. It lets us conclude that we can indeed say something about the input space X, based only on D. But only in a __probabilistic sense__ - by getting enough data, we can argue that it is very likely that $E_{in}(g) \approx E_{out}(g)$.

__The second answer is answered after we run the algorithm__ and see what the in-sample error 


#### Dichotomies, Growth func., Breakpoints

To find a meaningful replacement for M in the of a infinite hypothesis set, we first introduce __dichotomies__.

A dichotomy, is a mini hypothesis applied only to a finite set of points. So one dichotomy is a binary string of length N. And each hypothesis gives a single dichotomy $[h(x_1),...,h(x_N)]$. So the set of all dichotomies for a H-set is: $H(x_1,...x_N) = \{h(x_1),...,h(x_N) | h\in H\}$. Notice that many "similar" hypothesis do not contribute to the set, because they produce the same dichotomy. Notice also that since we are in a __binary classification setting__, there can be max $2^N$ dichotomies.

The replacement for M is going to be the __growth function__ (see proof below why this is okay):

$$m_H(N) = MAX_{x_1,...,x_N \in X} |H(x_1,...,x_N)|$$

i.e. the max number of dichotomies we can produce with any possible set of N points from X. The growth function is a finite number, so already there it is an improvement over M. The growth function captures the expressiveness of H on fixed data size.

We say that H __shatters__ $x_1,...x_N$ if $|H(x_1,...,x_N) = 2^N$. If this is the case then it means that the hypothesis set is complex enough to correctly divide any N points. If no data set of N points can be shattered by H then N is a __Break point__ for H. So a break point is the point at which you fail to get all possible dichotomies.

When replacing it with M we get the following hoeffding bound:

$$P[|E_{in}(h) - E_{out}(h)| > \epsilon] \le 2m_H(N)e^{-2\epsilon^2N}$$

__If we can then argue that $m_h(N)$ is a polynomial, then life is good, because the negative exponential (in N) in the hoeffding bound will always kill ANY polynomial factor if N becomes big enough__. This means that the RHS of Hoeffding becomes small which means that we can generalize. The following holds:

1) No break point $\implies m_H(N) = 2^N$

2) Break point exists $\implies m_H(N)$ is polynomial in N.


#### Proof that  growth function is polynomial in N if breakpoint exist

Define the quantity $B(N,k)$: Maximum number of dichotomies on N points, suck that no subset of k points can be shattered by these dichotomies (break point = k).

The plan is to bound the growth function by $B(N,k)$, and then bound that by a polynomial. i.e.:

$$m_H(N)\le B(N,k) \le some \ polynomial$$

#### Proof that we can swap M with the growth function

#### VC Dimension


### Bias variance tradeoff

### Regularization and validation